### Importing Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re, time,csv
from selenium.common.exceptions import NoSuchElementException

### Defining Function

In [7]:
def scrape(url):
    output='output.csv'
    
    file_handler=open(output,'w',encoding='utf8')
    
    writer=csv.writer(file_handler,lineterminator='\n')
    
    writer.writerow(["Reviewer's Name", 'Review Date', 'Review Text', 'Review Polarity']) #appends them to file in 1st row
    
    driver.get(url) # visit the reviews url for the given product
    
    while True: # keep going until there are no more review pages
                 
        driver.execute_script('window,scrollTo(0,document.body.scrollHeight)') #scroll down
       
        #get all the reviews in the page
        reviews=driver.find_elements_by_class_name("review_table_row")
        
        for review in reviews: # for each review
            
            name, date, text, polarity,='NA','NA','NA','NA' # Initialziing Attributes to NA
            
            # Name of Author
            try:
                nameBox = review.find_element_by_css_selector('[data-qa="review-critic-link"]')
            except:
                nameBox = None 
               
            if nameBox: name=nameBox.text  #retrieves text from titleBox
            
            # Date when review was posted
            try: 
                dateBox=review.find_element_by_css_selector('[data-qa="review-date"]')
            except:
                dateBox=None
                
            #box found, extract text
            if dateBox: date=dateBox.text
            
            
            # Review Comment
            try: # try to find the review box
                reviewBox=review.find_element_by_css_selector('[data-qa="review-text"]')
            except NoSuchElementException:
                reviewBox=None
           
            #box found, extract text
            if reviewBox: text = reviewBox.text
                
        
            try: # try to find the rating box
                #ratingBox=review.find_element_by_xpath("div[class^='review_icon icon small']")
                ratingBox = review.find_element_by_css_selector("div[class^='review_icon icon small']")
            except NoSuchElementException:
                ratingBox=None
                
            if ratingBox:
                ratingInfo=ratingBox.get_attribute('class') # get the class names mentioned in class attributes
                #print('rating' + ratingInfo)
                rating = ratingInfo.split(' ')   
                
                #rating=re.search(r'\b[sS]\f+|r+',ratingInfo)
                
                polarity = rating[-1]
            
            writer.writerow([ name, date, text, polarity])  # write the outputs obtained for current review to file
        
        #wait until the next Button loads
        nextButton=WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CSS_SELECTOR,'[data-qa="next-btn"]')))
        
        # In final page, the class attribute of next Button has one more class called 'hide' 
        # It hides the button from view
        if 'hide' in nextButton.get_attribute('class'): 
            break
            
        nextButton.click() # performs click on next button
        
        time.sleep(3)
    
    file_handler.close()

### Creating object of webdriver

In [10]:
driver = webdriver.Chrome('./chromedriver')

<ipython-input-10-c2e9cad52997>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [11]:
driver.maximize_window()

### Specifying URL and Calling Function

In [12]:
# URL of the RottenTomatoes Movie review

url='https://www.rottentomatoes.com/m/exodus_gods_and_kings/reviews'

In [13]:
scrape(url)

<ipython-input-7-04885745b62e>:17: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  reviews=driver.find_elements_by_class_name("review_table_row")
